### House price prediction with MLFLOW 

In [19]:
import pandas as pd 
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing)

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]]), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset:\n\nCalifornia Housing dataset\n-

In [20]:
# Preparing the dataset 
data = pd.DataFrame(housing.data, columns=housing.feature_names)
data['Price'] = housing.target
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## Train test split, Model Hyperparametor Tuning, MLFLOW Experiments

In [21]:
from urllib.parse import urlparse
## Independet and dependent features 
X = data.drop(columns=["Price"])
y = data["Price"]

In [22]:
## Hyperparameter tuning using Grid Searchcv 
from sklearn.model_selection import GridSearchCV

def hyperparameter_tuning(X_train, y_train, param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, 
                               scoring="neg_mean_square_error")
    grid_search.fit(X_train, y_train)
    return grid_search 

In [23]:
## split data into training and test sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

from mlflow.models import infer_signature
signature = infer_signature(X_train, y_train)

## Define the hyperparameter grid 
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 10, None],
    "min_sample_split": [2, 5],
    "min_sample_leaf": [1, 2]
}

## start the MLFLOW Experiments 

with mlflow.start_run():
    ## Perform hyperparameter tuning 
    gride_search = hyperparameter_tuning(X_train, y_train, param_grid)

    ## Get the best model 
    best_model = gride_search.best_estimator_

    ## Evaluate the best model 
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    ## Evaluate the model 
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    ## Log best parametors 
    mlflow.log_param("best_n_estimators", gride_search.best_params_['n_estimators'])
    mlflow.log_param("best_n_estimators", gride_search.best_params_['max_depth'])
    mlflow.log_param("best_n_estimators", gride_search.best_params_['min_sample_split'])
    mlflow.log_param("best_n_estimators", gride_search.best_params_['min_sample_leaf'])
    mlflow.log_metric("mse", mse)

    ## Tracking url 
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != 'file':
        mlflow.sklearn.load_model(best_model, "model", registered_model_name = "Best Randomforest Model")
    else:
        mlflow.sklearn.log_model(best_model, "model", signature=signature)

    print(f"Best Hyperparametors: {gride_search.best_params_}")
    print(f"Best Square Error: {mse}")
 

InvalidParameterError: The 'scoring' parameter of GridSearchCV must be a str among {'neg_brier_score', 'top_k_accuracy', 'r2', 'f1', 'average_precision', 'precision_samples', 'neg_mean_squared_log_error', 'f1_samples', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'completeness_score', 'f1_macro', 'jaccard_samples', 'recall', 'positive_likelihood_ratio', 'precision_micro', 'recall_weighted', 'recall_samples', 'adjusted_mutual_info_score', 'balanced_accuracy', 'neg_root_mean_squared_error', 'roc_auc_ovr', 'roc_auc_ovr_weighted', 'neg_median_absolute_error', 'precision', 'neg_mean_poisson_deviance', 'homogeneity_score', 'precision_macro', 'roc_auc_ovo_weighted', 'roc_auc', 'd2_absolute_error_score', 'neg_mean_gamma_deviance', 'neg_log_loss', 'neg_max_error', 'neg_mean_absolute_error', 'neg_root_mean_squared_log_error', 'jaccard_micro', 'jaccard', 'f1_micro', 'matthews_corrcoef', 'rand_score', 'v_measure_score', 'jaccard_macro', 'jaccard_weighted', 'recall_macro', 'adjusted_ran